In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pathlib
import random
import glob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers
import PIL

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

images_path = os.path.join(os.path.realpath('..'), "input", "kermany2018", "OCT2017 ")
images_path = pathlib.Path(images_path)

image_count = len(list(images_path.glob('*/*.jpeg')))
print(image_count)

batch_size = 128
img_height = 180
img_width = 180


test_ds = image_dataset_from_directory(
os.path.join(images_path, "test"),
  validation_split=0,
  seed=1232376,
  image_size=(img_height, img_width),
  batch_size=128128)

In [ ]:
val_ds = image_dataset_from_directory(
os.path.join(images_path, "val"),
  validation_split=0,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
train_ds = image_dataset_from_directory(
  os.path.join(images_path, "train"),
  validation_split=0,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
num_classes = 4

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(256, activation='relu'),
  layers.Dense(512, activation='relu'),
  layers.Dense(256, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

model.summary()

In [ ]:
model.evaluate(test_ds)

In [ ]:
# Calculate F1 score
from sklearn import metrics
# metrics.f1_score(y_actual, y_pred, labels=[0, 1, 2], average='macro')
y_pred =[]
y_true = []

i = 0
for batch, lbls in test_ds.as_numpy_iterator():
    prediction = model.predict_on_batch(batch)
    
    for i in range(len(prediction)):
        y_true.append(lbls[i])
        prediction_single = np.argmax(prediction[i])
        y_pred.append(prediction_single)
#     print(batch.shape)
#     print(lbls.shape)
print(y_true)
print(y_pred)
    
print(metrics.f1_score(y_true, y_pred, labels=[0, 1, 2, 3], average='macro'))